## Read in preprocessed data

In [24]:
import pandas as pd
import numpy as np
import time

np.random.seed(42)

X_train = pd.read_pickle('../data/X_train_v3.pkl')
y_train = pd.read_pickle('../data/y_train_v3.pkl')

_______________________________________________________________________________________________________________________________

## Build Models

In [25]:
#Import cross validation and optimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#Import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from multiprocessing import cpu_count

In [26]:
#Create a dictionary to hold models
models = {}

In [27]:
#Create function to add model and metrics to dictionary
def model_eval(model, name, X_train, y_train, cv):
    #Fit model
    model.fit(X_train, y_train)
    
    #Create predictions
    y_pred = model.predict(X_train)
    
    #Create cross validation scores
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=cpu_count())
    
    #Create cross validation predictions
    cv_pred = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=cpu_count())
    
    #Create metrics
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    roc_auc = roc_auc_score(y_train, y_pred)
    
    #Add metrics to dictionary
    models[name] = {'model': model,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'roc_auc': roc_auc,
                    'cv_score': cv_scores.mean(),
                    }
    
    #Print metrics
    print(name)
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print

In [28]:
#Instantiate models
rf_clf = RandomForestClassifier()
log_reg = LogisticRegression()
gbc = GradientBoostingClassifier()
svc_clf = SVC()
sgd_clf = SGDClassifier()
neigh_clf = KNeighborsClassifier()
dct_clf = DecisionTreeClassifier()
xgb_clf = XGBClassifier()

In [29]:
#Create a list of models
model_list = [rf_clf, log_reg, gbc, svc_clf, sgd_clf, neigh_clf, dct_clf, xgb_clf]

In [30]:
#iterate through list of models, evaluate, and add to dictionary
for model in model_list:
    model_eval(model, model.__class__.__name__, X_train, y_train, 10)

RandomForestClassifier
Accuracy:  0.9797979797979798
Precision:  0.990909090909091
Recall:  0.956140350877193
LogisticRegression
Accuracy:  0.8058361391694725
Precision:  0.7717041800643086
Recall:  0.7017543859649122
GradientBoostingClassifier
Accuracy:  0.8911335578002245
Precision:  0.9016393442622951
Recall:  0.804093567251462
SVC
Accuracy:  0.8372615039281706
Precision:  0.8272425249169435
Recall:  0.7280701754385965
SGDClassifier
Accuracy:  0.6464646464646465
Precision:  0.5279503105590062
Recall:  0.7456140350877193
KNeighborsClassifier
Accuracy:  0.8574635241301908
Precision:  0.8369905956112853
Recall:  0.7807017543859649
DecisionTreeClassifier
Accuracy:  0.9797979797979798
Precision:  0.9939024390243902
Recall:  0.9532163742690059
XGBClassifier
Accuracy:  0.9640852974186308
Precision:  0.9696969696969697
Recall:  0.935672514619883


In [31]:
#Print model with best roc_auc score
best_roc_auc = 0
for model in models:
    if models[model]['roc_auc'] > best_roc_auc:
        best_roc_auc = models[model]['roc_auc']
        best_model = model
        
print('Best Model: ', best_model)

Best Model:  RandomForestClassifier


In [32]:
#Print model with best cv score
best_cv_score = 0
for model in models:
    if models[model]['cv_score'] > best_cv_score:
        best_cv_score = models[model]['cv_score']
        best_model = model

print('Best Model: ', best_model)

Best Model:  GradientBoostingClassifier


In [33]:
#Print model with best f1 score
best_f1 = 0
for model in models:
    if models[model]['f1'] > best_f1:
        best_f1 = models[model]['f1']
        best_model = model

print('Best Model: ', best_model)

Best Model:  RandomForestClassifier


In [34]:
#Get confusion matrix for RandomForestClassifier
y_pred = models['RandomForestClassifier']['model'].predict(X_train)
confusion_matrix(y_train, y_pred)

array([[546,   3],
       [ 15, 327]])

In [35]:
#Get models from dict with roc_auc > 0.95
best_models = {}
for model in models:
    if models[model]['roc_auc'] > 0.95:
        best_models[model] = models[model]


In [36]:
#Print model and roc_auc
for model in best_models:
    print(model)
    print('roc_auc: ', best_models[model]['roc_auc'])
    print

RandomForestClassifier
roc_auc:  0.9753379350014381
DecisionTreeClassifier
roc_auc:  0.9747866935097307
XGBClassifier
roc_auc:  0.95872878918608


In [37]:
#Print model and cv_score
for model in models:
    print(model)
    print('cv_score: ', models[model]['cv_score'])
    print

RandomForestClassifier
cv_score:  0.8058676654182273
LogisticRegression
cv_score:  0.8002372034956304
GradientBoostingClassifier
cv_score:  0.8294506866416977
SVC
cv_score:  0.8226966292134831
SGDClassifier
cv_score:  0.7790511860174781
KNeighborsClassifier
cv_score:  0.8069912609238452
DecisionTreeClassifier
cv_score:  0.7878776529338326
XGBClassifier
cv_score:  0.8125967540574284


In [38]:
#Optimize RandomForestClassifier
start = time.time()
param_grid = [{'n_estimators': [3, 10, 30, 100, 300, 1000],
                'max_features': [2, 4, 6, 8, 10],
                'max_depth': [2, 4, 6, 8, 10],
                'max_leaf_nodes': [2, 4, 6, 8, 10, 20, 50],
                'n_jobs': [-1]}]

rf_clf = RandomForestClassifier()

grid_search = GridSearchCV(rf_clf, param_grid, cv=10, scoring='roc_auc', n_jobs=cpu_count())

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

stop = time.time()
total_time = stop - start

print(best_params)
print(total_time)


{'max_depth': 8, 'max_features': 8, 'max_leaf_nodes': 20, 'n_estimators': 10, 'n_jobs': -1}
618.9058589935303


In [39]:
#create a RandomForestClassifier with best params
start = time.time()
rf_clf = RandomForestClassifier(**best_params)

#fit the model
rf_clf.fit(X_train, y_train)

#evaluate the model
y_pred = rf_clf.predict(X_train)

rf_clf_classification_report = classification_report(y_train, y_pred)
rf_clf_confusion_matrix = confusion_matrix(y_train, y_pred)

stop = time.time()
total_time = stop - start

print(rf_clf_classification_report)
print(rf_clf_confusion_matrix)
print(total_time)

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       549
           1       0.88      0.75      0.81       342

    accuracy                           0.87       891
   macro avg       0.87      0.84      0.85       891
weighted avg       0.87      0.87      0.86       891

[[515  34]
 [ 85 257]]
0.038655757904052734


In [40]:
#Optimize RandomForestClassifier
start = time.time()
param_grid = [{'n_estimators': [5, 10, 20, 50, 100, 500, 1000, 5000],
                'max_features': [2, 4, 6, 8, 10],
                'max_depth': [2, 4, 6, 8, 10],
                'max_leaf_nodes': [2, 4, 6, 8, 10, 20, 50],
                'n_jobs': [-1]}]

rf_clf = RandomForestClassifier()

grid_search = GridSearchCV(rf_clf, param_grid, cv=10, scoring='roc_auc', n_jobs=cpu_count())

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(best_params)

KeyboardInterrupt: 

In [ ]:
#create a model with best params
rf_clf = RandomForestClassifier(**best_params)

#fit the model
rf_clf.fit(X_train, y_train)

#evaluate the model
y_pred = rf_clf.predict(X_train)

rf_clf_classification_report = classification_report(y_train, y_pred)
rf_clf_confusion_matrix = confusion_matrix(y_train, y_pred)
print(rf_clf_classification_report)
print(rf_clf_confusion_matrix)

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       549
           1       0.91      0.72      0.81       342

    accuracy                           0.87       891
   macro avg       0.88      0.84      0.85       891
weighted avg       0.87      0.87      0.86       891

[[525  24]
 [ 95 247]]
